# Methods to fit an overparameterized linear model

Basic algebra teaches us that a system of equations having N equations with N unknown parameters can be solved-- with a unique solution. But perfect solvability isn't the only situation. Define p as the number of unknown parameters.  Suppose there are N equations. Then there are two other cases:
* underparameterized: When p < N, then there are no solutions (or the system can be re-written as fewer equations).
* overparameterized: When p > N, there are many possible solutions.

Here I focus on the overparameterized case.  How do we choose a single solution?

# Setting up the example

But first, consider the simplest case of N equations with N unknowns, which is one equation with one unknown.  Here is an example:

`10 = 2 x`

That has one solution, namely `x = 5`.  (Note that we solve this by taking the product of 10 by the inverse of 2, which is 1/2.)

An underparameterize case would mean more equations than unknowns.  So let's add one more equation:

```
10 = 2 x
10 = 5 x
```

Clearly, no solution is perfect and we will need a way to select the "least bad" solution.  That way is usually linear regression with ordinary least squares (OLS).

Now consider the overparameterized case, which means there are more unknowns than equations, i.e. p > N. In the simplest case, there is one equation with TWO unknowns.  Let's add one more parameter to the above equation

`10 = 2 x + 4 y`

Clearly, there are many possible solutions and we will need a way to select one.  I will show a way by minimizing the L2 norm, equivalent to calculating the Moore-Penrose inverse.


The important point here is that the interpolation case is the only one with no freedom or wiggle room.  There is ONE solution and you are stuck with it, regardless of how well it generalizes.

# Libraries

In [21]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 14)

# Example underparameterized least-bad non-solutions


Recall our under-parameterized equations:

```
10 = 2 x
10 = 5 x
```

Let's consider some possible values of x and how far the equations would make them from the actual goal of 10.

In [28]:
x = np.arange(1.0, 6.0, 0.5)
df = pd.DataFrame.from_dict({'x':x, '2x':2*x, '2x-10':(2*x-10), '5x':5*x, '5x-10':(5*x-10), 'RSS (sum diff)':((2*x-10)**2+(5*x-10)**2)})
print(df.to_string(index=False))

  x   2x  2x-10   5x  5x-10  RSS (sum diff)
1.0  2.0   -8.0  5.0   -5.0           89.00
1.5  3.0   -7.0  7.5   -2.5           55.25
2.0  4.0   -6.0 10.0    0.0           36.00
2.5  5.0   -5.0 12.5    2.5           31.25
3.0  6.0   -4.0 15.0    5.0           41.00
3.5  7.0   -3.0 17.5    7.5           65.25
4.0  8.0   -2.0 20.0   10.0          104.00
4.5  9.0   -1.0 22.5   12.5          157.25
5.0 10.0    0.0 25.0   15.0          225.00
5.5 11.0    1.0 27.5   17.5          307.25


So if we chose the lowest-RSS value for x, it would be x=2.5, with an RSS of 31.25.

OLS stands for "ordinary least squares" and this method of picking an under-parameterized solution can sometimes generalize well.  It equivalent to root mean square but without the root-- giving the same ordering though.

I don't assess generalization error here.  This was just a train-only error, not including a test set.

# Example overparameterized solutions

We consider some possible values of x and determine what y would match those.  Note that because all of these solutions lead to a value of 10, there is no error and  RSS is always zero, so we cannot use the RSS to select one.

In [26]:
x = np.arange(0.5, 1.3, 0.1)
df = pd.DataFrame.from_dict({'x':x, 'y':(10-2*x)/4})
df['RSS'] = [0] * len(x)
print(df.to_string(index=False))

  x    y  RSS
0.5 2.25    0
0.6 2.20    0
0.7 2.15    0
0.8 2.10    0
0.9 2.05    0
1.0 2.00    0
1.1 1.95    0
1.2 1.90    0


## Choosing a solution with the min L2 norm

One possible way to choose a solution is to choose the one that minimizes the l2 norm of the parameters, meaning `x^2 + y^2`.  It is possible to solve this analystically, using the Moore-Penrose inverse, the matrix equivalent of taking the inverse of a number (1/2 vs. 2).

But here let's skip the math and just calculate the L2 values for our possible solutions and choose the minimum.

In [24]:
df["L2"] = df["x"]**2 + df["y"]**2
print(df.to_string(index=False))

  x    y     L2
0.5 2.25 5.3125
0.6 2.20 5.2000
0.7 2.15 5.1125
0.8 2.10 5.0500
0.9 2.05 5.0125
1.0 2.00 5.0000
1.1 1.95 5.0125
1.2 1.90 5.0500


The minimum L2, namely of 5.00, is attained at:
`(x=1.0, y=2.0)`

So let's choose that solution.

Just for reference, let me also add the residual sum of squares (RSS) to the table.

In [25]:
df = pd.DataFrame.from_dict({'x':x, 'y':(10-2*x)/4})
df["RSS"] = [0] * len(df)
df["L2"] = df["x"]**2 + df["y"]**2
print(df.to_string(index=False))

  x    y  RSS     L2
0.5 2.25    0 5.3125
0.6 2.20    0 5.2000
0.7 2.15    0 5.1125
0.8 2.10    0 5.0500
0.9 2.05    0 5.0125
1.0 2.00    0 5.0000
1.1 1.95    0 5.0125
1.2 1.90    0 5.0500
